In [1]:
# Prerequisite :
# 1.) Create AdaptReady Folder under C: drive.
# 2.) Paste hospital_names.txt file under C:\AdaptReady\
# 3.) Paste AdaptReady.pynb under C:\AdaptReady\ and run the code in jupyter notebook.
# Note : I have also attached the screenshot for your reference.

# Import all the necessary libraries.

import numpy as np
import pandas as pd
import sklearn
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import LambdaCallback
from keras.models import load_model
import requests

Using TensorFlow backend.


In [2]:
# Open and read the text file i.e. hospital_names.txt.
# Copy the contents of text file in "text" variable and then close the text file.

with open("C:\\AdaptReady\\hospital_names.txt", ) as text_file:
    text = text_file.read()

In [3]:
# Split the text file by "\n" to create the list of all the hospital names.

hospital_names = text.split("\n")

In [4]:
# Remove all the punctuations from the hospital names.
# Create the list of all the hospital names without punctuations.
# list of hospital_names_without_punctuations is in lowercase format.

hospital_names_without_punctuations = []

for i in range(0, len(hospital_names)):
    for char in hospital_names[i]:
        if char in """"!@#$%^&*()_+-={}[]|"'\:;<>,.?/-+`â~€“""":
            hospital_names[i] = hospital_names[i].replace(char, "") 
    hospital_names_without_punctuations.append(hospital_names[i].lower())

In [5]:
# Remove repeated hospital names and add period after each hospital name.

hospital_names_without_punctuations = list(map(lambda s: s + '.', hospital_names_without_punctuations))

In [6]:
# Map all the lowercase alphabets with unique integer and later mapped with their ASCII code.
# e.g. : a - z is mapped with 1 - 26 uniquely.
# space is mapped with 0
# period is mapped with 27
# Create two numpy array of zeros i.e. X and Y  of mentioned length as given parameters.
# Replace the zero under X and Y by their corresponding value of alphabet used in hospital_names_without_punctuations.

char_to_index = dict((chr(i+96), i) for i in range(1,27))
char_to_index[' '] = 0
char_to_index['.'] = 27

index_to_char = dict( (i, chr(i+96)) for i in range(1,27))
index_to_char[0] = ' '
index_to_char[27] = '.'

max_char = len(max(hospital_names_without_punctuations, key=len))
m = len(hospital_names_without_punctuations)
char_dim = len(char_to_index)

X = np.zeros((m, max_char, char_dim))
Y = np.zeros((m, max_char, char_dim))

for i in range(m):
    name = list(hospital_names_without_punctuations[i])
    for j in range(len(name)):
        X[i, j, char_to_index[name[j]]] = 1
        if j < len(name)-1:
            Y[i, j, char_to_index[name[j+1]]] = 1

In [7]:
# Created a sequential model using LSTM network.

model = Sequential()
model.add(LSTM(128, input_shape=(max_char, char_dim), return_sequences=True))
model.add(LSTM(128, input_shape=(max_char, char_dim), return_sequences=True))
model.add(LSTM(128, input_shape=(max_char, char_dim), return_sequences=True))
model.add(Dense(char_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# number_of_iterations_run(epoch, _) is a method through which we came to know how many iterations ran till yet.

def number_of_iterations_run(epoch, _):
 
    if epoch % 100 == 0:
        
        print('Number of iterations completed : %d' % epoch)
      
name_generator = LambdaCallback(on_epoch_end = number_of_iterations_run)

In [9]:
# Train the model.

model.fit(X, Y, batch_size=64, epochs=1000, callbacks=[name_generator], verbose=0)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Number of iterations completed : 0
Number of iterations completed : 100
Number of iterations completed : 200
Number of iterations completed : 300
Number of iterations completed : 400
Number of iterations completed : 500
Number of iterations completed : 600
Number of iterations completed : 700
Number of iterations completed : 800
Number of iterations completed : 900


In [10]:
# Save the model.

model.save("C:\\AdaptReady\\trained_model.h5")

In [11]:
# Load the model.

model = load_model("C:\\AdaptReady\\trained_model.h5")

In [12]:
# predict_hospital_name(model, seed) is a method i.e. used to predict the hopital names using our trained_model.h5 file.
# We can give the "seed" as parameter i.e. characters or name from which we want to start our predicted hospital name.
# predict_hospital_name(model, seed) method is a method used to predict the next characters one after the another after the given seed or any character till it reaches to period.

def predict_hospital_name(model, seed):
    name = [seed]
    x = np.zeros((1, max_char, char_dim))
    end = False
    i = 0
    
    while end==False:
        probs = list(model.predict(x)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(char_dim), p=probs)
        if i == max_char-2:
            character = '.'
            end = True
        else:
            character = index_to_char[index]
        name.append(character)
        x[0, i+1, index] = 1
        i += 1
        if character == '.':
            end = True
    
    print(''.join(name))

# Predict Hospital Name Using predict_hospital_name(model, seed) method by giving seed parameter in the form of string.

In [13]:
predict_hospital_name(model, "amity")

amityorthwest texas healthcare system.


In [14]:
predict_hospital_name(model, "jan")

janalines medical center.


In [15]:
predict_hospital_name(model, "takshila")

takshilaorthwest community hospital.


In [16]:
predict_hospital_name(model, "hen")

henano regional hospital.


In [17]:
predict_hospital_name(model, "guru")

guruidan goondothomaranth medical center.


In [19]:
predict_hospital_name(model, "")

orthwest medical center.


In [21]:
predict_hospital_name(model, "a")

aano valley medical center west valley campus.


# THANK YOU